In [107]:
import pandas as pd, numpy as np, re

from numpy import asarray, power

In [118]:
class Data:
    def __init__(self, data_path, fname_arr, ref_gene, treated=True):
        self.data_path = data_path
        self.fname_arr = fname_arr
        self.ref_gene  = ref_gene
        self.treated   = treated
    
    def load_csv(self):
        # Reads multiple csv files. Input path of file and
        # array of filenames assuming they are in the same folder. Output a list of dataframes.
        df_list = []
        for i, df_name in enumerate(self.fname_arr):
            fname  = self.data_path+df_name+'.csv'
            df = pd.read_csv(fname, header=0)
            df['Plate'] = i+1
            df_list.append(df)
        return df_list
    
    def raw_data(self):
        return self.load_csv()
    
    #--------------------------------------------------------------------------------
    # Just a bunch of housekeeping functions to tidy and prepare raw LightCycler data
    
    def tidy_each_experiment(self):
        df_list = self.load_csv()
        tidied  = []
        for df in df_list:
            t_df = self.tidy(df)
            tidied.append(t_df)
        return tidied
        
    def tidy(self,df):
        
        df = self.trim_all_columns(df)
        
        if 'Plate' not in df.columns: df['Plate'] = 'NaN'
        
        df = self.remove_columns(df)
        df = self.rename_columns(df)
        df = df.replace(regex=r'-', value=0)
        
        df['Cq'] = pd.to_numeric(df['Cq'])
        
        if self.treated: df['Treatment'] = df['Treatment'].str.title()

        df = self.add_columns(df)

        # Set value to 0 if Cq is >= 40
        df['Cq'].where(df['Cq'] < 40, 0, inplace=True)

        # Reorder columns
        columns_titles = ['Sample','Bio Rep', 'Target', 'Cq', 'Cq Mean', 'Replicate Group', 'Condition', 'Treatment', 'Plate']
        df = df[columns_titles]
        
        return df
    
    def trim_all_columns(self,df):
        """
        Trim whitespace from ends of each value across all series in dataframe
        """
        
        #Remove whitespace around column headers if any.
        df.rename(columns=lambda x: x.strip(),inplace = True)

        trim_strings = lambda x: x.strip() if isinstance(x, str) else x
        return df.applymap(trim_strings)
    
    def remove_columns(self,df):
        #Remove extraneous columns generated by LightCycler export. Columns that are retained are commented out.
        df = df.drop(['Color',
         'Position',
        #  'Sample Name',
        #  'Gene Name',
        #  'Condition Name',
        #  'Cq',
        #  'Cq Mean',
         'Cq Error',
         'Excluded',
         'Sample Type',
         'Sample Type RQ',
         'Gene Type',
         'Condition Type',
        #  'Replicate Group',
         'Ratio',
         'Ratio Error',
         'Normalized Ratio',
         'Normalized Ratio Error',
         'Scaled Ratio',
         'Scaled Ratio Error',
         'Dye',
         'Edited Call',
         'Failure',
         'Slope',
         'EPF',
         'Notes',
         'Sample Prep Notes',
         'Number'
         #,'Plate'
          ], axis=1)
        return df

    def rename_columns(self,df):
        """
        This function renames columns 'Sample Name', 'Gene Name' to 'Sample' and 'Target' respectively.
        :return: Updated df
        :rtype: DataFrame
        """
        df       = df.rename(columns={'Sample Name':'Sample','Gene Name': 'Target', 'Condition Name': 'Treatment'})
        return df
    
    def add_columns(self,df):
        # Add column to define condition
        sample_condition = asarray([re.sub("[^A-Z\d]", "", re.search("^[^_]*", i).group(0).upper()) for i in df['Sample']])
        df['Condition'] = sample_condition

        # Add column to define bio replicate number
        sample_bio_rep = asarray([i[-1:] for i in df['Sample']])
        df['Bio Rep'] = sample_bio_rep
        
        return df
    
    #--------------------------------------------------------------------------------
    
    '''
    1. average tech reps
    2. normalise to bactin - dct from each sample (bio duplicate) - sample control
    3. get relative expression for each sample
    4. take mean of rq of bio replicates
    5. fold change relative to biological control (1A?) - i.e. normalise again to day 3 1A non gravel - plot this.

    '''
    
    def analyse(self):
        """Description 
        :param DataFrame sample_frame: A sample data frame.
        :param string ref_target: A string matching an entry of the Target column; reference gene;
            the target to use as the reference target (e.g. 'BACTIN')
        :return: A DataFrame with columns: Sample, Target, Age, DeltaCq, and Rel Exp.
        :rtype: DataFrame
        """
        df       = self.tidy_each_experiment()
        ref_gene = self.ref_gene
        relevant_cols = ['Sample', 'Cq', 'Condition', 'Bio Rep']
        relevant_grps = ['Condition', 'Sample', 'Bio Rep']
        if self.treated: relevant_cols.append('Treatment'); relevant_grps.append('Treatment')
        
        if isinstance(df,list): df = df[0]
        
        ref_target_mean_by_sample, ref_target_mean_by_treat = self.get_ref_data(df, relevant_cols, relevant_grps)
        
        ref_sample_grouped_by_age = df.groupby(relevant_grps)
        
        # define conditions and initiate empty array and df
        unique_cond = df.Condition.unique()
        rq_df       = pd.DataFrame({
                                   'Sample': [],
                                   'Target': [],
                                   'Age': [],
                                   'RQ': [],
                                   'Treatment': []
                                  })
        
        # iterate through unique 'conditions' i.e. ages and NEG, calculate relative quantity, and 
        for condition, group in ref_sample_grouped_by_age:
            for age in unique_cond:
                if age == condition[0]:
                    sample         = condition[1]
                    biorep         = group['Bio Rep'].unique()[0]
                    treatment      = group['Treatment'].unique()[0]
                    
                    ref_mean_cq    = self.get_ref_mean(ref_target_mean_by_sample, age, biorep, treatment)['Cq']
                    sample_mean_cq = self.amean_cq(group['Cq']) # arithmetic mean of tech reps
                    
                    # Normalise to reference gene and 
                    RQ = self.ddcq(ref_mean_cq, sample_mean_cq)
                    
                    rq_df = rq_df.append({
                                           'Sample'   : sample,
                                           'Target'   : group['Target'].unique()[0],
                                           'Age'      : age,
                                           'RQ'       : RQ,
                                           'Treatment': treatment
                                         },ignore_index=True)
        
        rq_df = rq_df.sort_values(['Treatment', 'Target'])
        
        # normalisation factor is the experimentally relevant group such as untreated control
        # or a particular target gene that your final results will be relative to
        norm_factor = rq_df.groupby(['Target', 'Age', 'Treatment']).agg(self.amean_cq)
        print(norm_factor, 'need to pick what we should norm to')
        
        
        # RQ/nf
        norm_expression_per_sample = 0
        
        # Bio group expression gmean of 
        
        
        
        
        
        
        return rq_df
    
    def get_ref_data(self,df,relevant_cols,relevant_grps):
        
        # select relevant columns from reference gene data and find mean of technical replicates
        ref_target_df             = df.loc[df['Target'] == self.ref_gene, relevant_cols]
        ref_target_grouped_by_age = ref_target_df.groupby(relevant_grps)
        ref_target_mean_by_sample = ref_target_grouped_by_age.agg(self.amean_cq)

        ref_target_mean_by_treat  = ref_target_mean_by_sample.groupby(['Treatment','Condition']).agg(self.amean_cq)
        
        return ref_target_mean_by_sample, ref_target_mean_by_treat
    
    def get_ref_mean(self, ref_mean_df, age, biorep, treatment):
        
        ref_mean_by_age = ref_mean_df.loc[[age, 'average_cq']].reset_index()
        ref_mean_cq     = ref_mean_by_age[(ref_mean_by_age['Bio Rep'] == biorep) & (ref_mean_by_age['Treatment'] == treatment)]
        return ref_mean_cq
    
    def ddcq(self, ref_mean_cq, sample_mean_cq):
        dcq = ref_mean_cq - sample_mean_cq
        ddcq = float(power(2, dcq))
        return ddcq
    
    def amean_cq(self,seq):
        amean_cq = np.mean(seq)
        return amean_cq
    
    



In [119]:
path_200117  = "../../Analyses/200117_qPCR/"
fname_200117 = ["200117_WT58_gravel_non_gravel_pt1", "200117_WT58_gravel_non_gravel_pt2"]
data_200117  = Data(path_200117, fname_200117, 'BACTIN')
# data.tidy_each_experiment()


path_fake  = "../../Analyses/fake/"
fname_fake = ["fake3"]
data_fake  = Data(path_fake, fname_fake, 'BACTIN')
# df = data_fake.load_csv()[0]
# data_fake.remove_columns(df)

data_fake.analyse()

# fake_df = data_fake.tidy_each_experiment()[0]

                                RQ
Target Age Treatment              
BACTIN 3   Gravel         1.000000
           Non Gravel     1.000000
       5   Gravel         1.000000
           Non Gravel     1.000000
       7   Gravel         1.000000
           Non Gravel     1.000000
       NEG Gravel         1.000000
           Non Gravel     1.000000
GRIN1A 3   Gravel         0.824951
           Non Gravel  3254.997354
       5   Gravel         1.587401
           Non Gravel     0.161396
       7   Gravel         5.041637
           Non Gravel     2.793701
       NEG Gravel         1.000000
           Non Gravel     1.000000


,Sample,Target,Age,RQ,Treatment
4,3_BACTIN_1,BACTIN,3,1.000000,Gravel
6,3_BACTIN_2,BACTIN,3,1.000000,Gravel
12,5_BACTIN_1,BACTIN,5,1.000000,Gravel
14,5_BACTIN_2,BACTIN,5,1.000000,Gravel
20,7_BACTIN_1,BACTIN,7,1.000000,Gravel
22,7_BACTIN_2,BACTIN,7,1.000000,Gravel
28,NEG_BACTIN_1,BACTIN,NEG,1.000000,Gravel
30,NEG_BACTIN_2,BACTIN,NEG,1.000000,Gravel
0,3_1A_1,GRIN1A,3,0.062500,Gravel
2,3_1A_2,GRIN1A,3,1.587401,Gravel
